# Convolutional Neural Network (CNN)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
%matplotlib inline
%config InlineBackend.figure_format='retina'
print ("PyTorch version:[%s]."%(torch.__version__))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print ("device:[%s]."%(device))

PyTorch version:[1.7.0+cu101].
device:[cpu].


### Dataset

In [3]:
from torchvision import datasets,transforms
mnist_train = datasets.MNIST(root='./data/',train=True,transform=transforms.ToTensor(),download=True)
mnist_test = datasets.MNIST(root='./data/',train=False,transform=transforms.ToTensor(),download=True)
print ("mnist_train:\n",mnist_train,"\n")
print ("mnist_test:\n",mnist_test,"\n")
print ("Done.")

mnist_train:
 Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data/
    Split: Train
    StandardTransform
Transform: ToTensor() 

mnist_test:
 Dataset MNIST
    Number of datapoints: 10000
    Root location: ./data/
    Split: Test
    StandardTransform
Transform: ToTensor() 

Done.


### Data Iterator

In [4]:
BATCH_SIZE = 256
train_iter = torch.utils.data.DataLoader(mnist_train,batch_size=BATCH_SIZE,shuffle=True,num_workers=1)
test_iter = torch.utils.data.DataLoader(mnist_test,batch_size=BATCH_SIZE,shuffle=True,num_workers=1)
print ("Done.")

Done.


### Define Model

In [22]:
class ConvolutionalNeuralNetworkClass(nn.Module):
    """
        Convolutional Neural Network (CNN) Class
    """
    def __init__(self,name='cnn',xdim=[1,28,28],ksize=(3,3),hdims=[32,64],ydim=10):
        super(ConvolutionalNeuralNetworkClass,self).__init__()
        self.name = name
        self.xdim = xdim
        self.ksize = ksize
        self.hdims = hdims
        self.ydim = ydim

        self.layers = []
        prev_dim = self.xdim[0]
        for l_idx in range(len(self.hdims)): # for each hidden layer
            hdim = self.hdims[l_idx]
            self.layers.append(
                nn.Conv2d(in_channels=prev_dim,
                          out_channels=hdim,
                          kernel_size=self.ksize,
                          stride=(1,1),
                          padding=self.ksize//2))
            self.layers.append(nn.BatchNorm2d(hdim))
            self.layers.append(nn.ReLU(True)) 
            self.layers.append(nn.MaxPool2d(kernel_size=(2,2), stride=(2,2)))
            prev_dim = hdim
        self.layers.append(nn.Flatten())
        fdim = int(self.xdim[1]/ 2**len(self.hdims))
        self.layers.append(nn.Linear(fdim*fdim*prev_dim,self.ydim,bias=True))

        self.net = nn.Sequential(*self.layers)
        self.init_param() # initialize parameters
        
    def init_param(self):
        for m in self.modules():
            if isinstance(m,nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m,nn.BatchNorm2d):
                nn.init.constant_(m.weight,1)
                nn.init.constant_(m.bias,0)
            elif isinstance(m,nn.Linear):
                nn.init.kaiming_normal_(m.weight)
                nn.init.zeros_(m.bias)
            
    def forward(self,x):
        return self.net(x)

C = ConvolutionalNeuralNetworkClass(
    name='cnn',xdim=[1,28,28],ksize=3,hdims=[32,64],ydim=10).to(device)
loss = nn.CrossEntropyLoss()
optm = optim.Adam(C.parameters(),lr=1e-3)
print ("Done.")

Done.


### Simple Forward Path of the CNN Model

In [23]:
np.set_printoptions(precision=3)
torch.set_printoptions(precision=3)
x_numpy = np.random.rand(2,1,28,28)
x_torch = torch.from_numpy(x_numpy).float().to(device)
y_torch = C.forward(x_torch) # forward path
y_numpy = y_torch.detach().cpu().numpy() # torch tensor to numpy array
print ("x_numpy:\n",x_numpy)
print ("x_torch:\n",x_torch)
print ("y_torch:\n",y_torch)
print ("y_numpy:\n",y_numpy)

x_numpy:
 [[[[0.445 0.854 0.277 ... 0.912 0.755 0.683]
   [0.822 0.746 0.004 ... 0.88  0.003 0.059]
   [0.465 0.287 0.541 ... 0.438 0.783 0.605]
   ...
   [0.342 0.324 0.616 ... 0.736 0.501 0.654]
   [0.965 0.936 0.432 ... 0.366 0.047 0.573]
   [0.589 0.759 0.162 ... 0.117 0.016 0.515]]]


 [[[0.88  0.745 0.693 ... 0.746 0.905 0.355]
   [0.119 0.574 0.884 ... 0.263 0.603 0.806]
   [0.448 0.076 0.791 ... 0.726 0.089 0.248]
   ...
   [0.134 0.116 0.989 ... 0.539 0.15  0.655]
   [0.631 0.404 0.831 ... 0.9   0.51  0.903]
   [0.557 0.138 0.008 ... 0.48  0.815 0.946]]]]
x_torch:
 tensor([[[[0.445, 0.854, 0.277,  ..., 0.912, 0.755, 0.683],
          [0.822, 0.746, 0.004,  ..., 0.880, 0.003, 0.059],
          [0.465, 0.287, 0.541,  ..., 0.438, 0.783, 0.605],
          ...,
          [0.342, 0.324, 0.616,  ..., 0.736, 0.501, 0.654],
          [0.965, 0.936, 0.432,  ..., 0.366, 0.047, 0.573],
          [0.589, 0.759, 0.162,  ..., 0.117, 0.016, 0.515]]],


        [[[0.880, 0.745, 0.693,  ..., 0.

### Check Parameters

In [25]:
np.set_printoptions(precision=3)
n_param = 0
for p_idx,(param_name,param) in enumerate(C.named_parameters()):
    if param.requires_grad:
        param_numpy = param.detach().cpu().numpy()
        n_param += len(param_numpy.reshape(-1))
        print ("[%d] name:[%s] shape:[%s]."%(p_idx,param_name,param_numpy.shape))
        print ("    val:%s"%(param_numpy.reshape(-1)[:5]))
print ("Total number of parameters:[%s]."%(format(n_param,',d')))

[0] name:[net.0.weight] shape:[(32, 1, 3, 3)].
    val:[0.864 0.095 0.515 0.272 0.29 ]
[1] name:[net.0.bias] shape:[(32,)].
    val:[0. 0. 0. 0. 0.]
[2] name:[net.1.weight] shape:[(32,)].
    val:[1. 1. 1. 1. 1.]
[3] name:[net.1.bias] shape:[(32,)].
    val:[0. 0. 0. 0. 0.]
[4] name:[net.4.weight] shape:[(64, 32, 3, 3)].
    val:[-0.003 -0.073 -0.049 -0.039  0.034]
[5] name:[net.4.bias] shape:[(64,)].
    val:[0. 0. 0. 0. 0.]
[6] name:[net.5.weight] shape:[(64,)].
    val:[1. 1. 1. 1. 1.]
[7] name:[net.5.bias] shape:[(64,)].
    val:[0. 0. 0. 0. 0.]
[8] name:[net.9.weight] shape:[(10, 3136)].
    val:[-0.003  0.03   0.017  0.034  0.005]
[9] name:[net.9.bias] shape:[(10,)].
    val:[0. 0. 0. 0. 0.]
Total number of parameters:[50,378].


### Check Layers